In [2]:
import sys
sys.path.append("/tesis/repos/tsm-pipeline/code")

%load_ext autoreload
%autoreload 2

In [4]:
from scripts.xxx import run_tsm, ExperimentSettings
import docker

docker_client = docker.from_env()
defaults = {
    "bash_config_path": "/tesis/repos/tsm-pipeline/code/scripts/config.sh",
    "o11y_db_dir": "/tesis/repos/tsm-pipeline/code/tsm.db",
}

El experimento de abajo es una versión de prueba para ver tanto que funciona bien la versión dockerizada y su wrapper en python, como tainted path.

In [34]:
test_experiment = ExperimentSettings(
    name="jupyter_test_experiment",
    query_type="path",
    bash_config_path="/tesis/repos/tsm-pipeline/code/scripts/config.sh",
    project_list_file="/tesis/repos/tsm-pipeline/experiments/tesis/tainted_path/test_4.txt",
    results_dir="/tesis/tmp/results/tainted_path_test",
    o11y_db_dir="/tesis/repos/tsm-pipeline/code/tsm.db",
)

run_tsm(docker_client, test_experiment, tail_logs=True)

b'[INFO\t2022-12-30 22:10:15,841] root\tGlobal config dump:\n'
b"{'config': {'codeQLExecutable': '/cli/codeql',\n"
b"            'codeQLSourcesRoot': '/ql',\n"
b"            'logsDirectory': '/bigtmp/log',\n"
b"            'resultsDirectory': '/results',\n"
b"            'workingDirectory': '/bigtmp/wd',\n"
b"            'worseLibSearchPath': '/worse_lib'}}\n"
b'[INFO\t2022-12-30 22:10:15,842] root\tResults folder: /results\n'
b'[INFO\t2022-12-30 22:10:16,416] root\tProject cache enabled with dir: /dbs\n'
b'2022-12-30 22:10:16,422 - INFO - starting new tracker factory at path /data/tracking.db\n'
b'[INFO\t2022-12-30 22:10:16,422] o11y\tstarting new tracker factory at path /data/tracking.db\n'
b'[INFO\t2022-12-30 22:10:16,429] root\tDumping project list for tracking purposes:\n'
b'1\t/dbs/2.5.2/ASCOT/dashboardjs/352a8b6710d6f2ddfd42ddfa0266af65c9c23ebe\n'
b'2\t/dbs/2.5.2/alphagov/submit-prototype-kit/6ce9e75078ea7965cb9429446cca8c5765817d7c\n'
b'3\t/dbs/2.5.2/RuffApps/Apps/90e13d23b0593

In [36]:
!cat /tesis/tmp/results/tainted_path_test/averaged-results.csv

"(parameter 0 (member define (global)))","snk",1.0000000000
"(parameter 0 (member readFile *))","snk",1.0000000000
"(parameter 0 (member readFileSync *))","snk",1.0000000000
"(parameter 0 (member readdir *))","snk",1.0000000000
"(parameter 0 (member require (global)))","snk",1.0000000000
"(parameter 0 (member stat *))","snk",1.0000000000
"(parameter 0 (member statSync *))","snk",1.0000000000

Ahora necesito ver por qué no encuentra resultados para 80 proyectos de `NoSQL`. Yo creo que debería ser capaz de encontrar.

In [6]:
nosql_test = ExperimentSettings(
    name="nosql_test_2",
    query_type="nosql",
    project_list_file="/tesis/repos/tsm-pipeline/experiments/tesis/test_nosql_5.txt",
    results_dir="/tesis/tmp/results/nosql_test_2",
    **defaults
)

run_tsm(docker_client, nosql_test, block=True)

2022-12-30 22:53:01,120 - INFO - creating /tesis/tmp/results/nosql_test_2 dir. Omitting error if already existing
2022-12-30 22:53:01,123 - INFO - creating /tesis/tmp/log dir. Omitting error if already existing


In [ ]:
!cat /tesis/tmp/results/nosql_test_2